In [24]:
from typing import Any, Dict
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, load_prompt
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache
from langchain.llms.loading import load_llm

set_llm_cache(InMemoryCache())
# set_debug(True)
# json_prompt = load_prompt("./prompt.json")

chat = ChatOpenAI(
    temperature = 0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

llm = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model="gpt-3.5-turbo-16k"
)

llm.save("model.json")
# llms2= load_llm("model.json")

examples = [
    {
        "movie" : "Dune",
        "answer" : """
        --Information about the movie--
        Director : Denis Villeneuve
        Main Cast : Timothée Chalamet as Paul Atreides & Rebecca Ferguson as Lady Jessica Atreides
        Budget : 165 million USD
        Box office revenue : 434.8 million USD 
        Genre : Sci-fi, Adventure, Action, Fantasy, Drama.
        Summary : A noble family becomes embroiled in a war for control over the galaxy's most valuable asset while its heir becomes troubled by visions of a dark future.
        """,
    },
    {
        "movie" : "Dune",
        "answer" : """
        --Information about the movie--
        Director : David Fincher
        Main Cast : Brad Pitt as Tyler Durden & Edward Norton as the Narrator
        Budget : 63 million USD
        Box office revenue : 101 million USD
        Genre : Drama, Thriller
        Summary : An insomniac office worker and a devil-may-care soap maker form an underground fight club that evolves into something much, much more.
        """
    }
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self,examples):
        self.examples = examples
    def add_example(self, example):
        self.examples.append(example)
    def select_examples(self, input_variables):
        from random import choice
        return [choice(examples)]
        

example_template = """
    Human: {movie}
    AI: {answer}
"""
example_prompt = PromptTemplate.from_template(example_template)

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=200,
)

example_random_selector = RandomExampleSelector(examples=examples)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: {movie}?",
    input_variables=['movie']
)


# prompt.format(movie="Parasite")

# chain = prompt | chat

# chain.invoke({'movie' : 'Joker'})

example_prompt2 = ChatPromptTemplate.from_messages([
    ("human", "{movie}"),
    ("ai", "{answer}")
])

prompt2 = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt2,
    example_selector=example_random_selector,
)


final_prompt = ChatPromptTemplate.from_messages([
    ("system", "you can only speak in korean"),
    prompt2,
    ('human', "{movie}")
])

# final_prompt.format(movie="Parasite")



chain2 = final_prompt | chat

chain2.invoke({'movie': 'Parasite'}) 






--영화 정보--
감독 : 봉준호
주연 : 송강호, 이선균, 조여정, 최우식
예산 : 약 11억 원
관객 수 : 1000만 명 이상
장르 : 블랙 코미디, 스릴러
줄거리 : 빈부격차가 심한 사회에서 살아가는 한 가족이 부자 가정에 침입하면서 벌어지는 사건들을 그린 작품.

AIMessageChunk(content='--영화 정보--\n감독 : 봉준호\n주연 : 송강호, 이선균, 조여정, 최우식\n예산 : 약 11억 원\n관객 수 : 1000만 명 이상\n장르 : 블랙 코미디, 스릴러\n줄거리 : 빈부격차가 심한 사회에서 살아가는 한 가족이 부자 가정에 침입하면서 벌어지는 사건들을 그린 작품.')